# Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
import re
import time
import pandas as pd
from google.colab import userdata
import google.generativeai as genai

In [2]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

fatal: destination path 'Financial_Sentiment_LLM' already exists and is not an empty directory.


In [3]:
# Global Variables
CATEGORIES = [
        "Finance",
        "Production",
        "Reserves / Exploration / Acquisitions / Mergers / Divestments",
        "Environment / Regulatory / Geopolitics",
        "Alternative Energy / Lower Carbon",
        "Oil Price / Natural Gas Price / Gasoline Price"]
SENTIMENT_RESULTS_FILE_PATH = '/content/Financial_Sentiment_LLM/03_Sentiment_Analysis/Sentiment_Analysis_Results.csv'

# Import Data

In [4]:
# Import all cleaned data files
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10126, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10121,Earnings Call Q&A,EQ-338,XOM,Feb-02-2021,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10122,Earnings Call Q&A,EQ-339,COP,Feb-02-2021,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10123,Earnings Call Q&A,EQ-340,EOG,May-03-2019,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10124,Earnings Call Q&A,EQ-341,SHEL,May-02-2019,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10125,Earnings Call Q&A,EQ-342,COP,Apr-30-2019,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


# Find or Create Results CSV

In [5]:
# Determine if the Sentiment Analysis Results file already exists
file_exists = os.path.isfile(SENTIMENT_RESULTS_FILE_PATH)

# Print the result
if file_exists:
    print(f"The file exists in the current directory.")
else:
    print(f"The file does not exist in the current directory.")

The file exists in the current directory.


In [6]:
# Create an empty file if the file does not exist
if not file_exists:
    # Copy text df and drop the text and headline column due to size
    empty_sentiment_df = text_df.copy()
    empty_sentiment_df = empty_sentiment_df.drop(['Article Text', 'Article Headline'], axis=1)

    for category in CATEGORIES:
        empty_sentiment_df[category] = ""

    # Display results
    display(empty_sentiment_df.head())

    # Save as CSV
    empty_sentiment_df.to_csv(SENTIMENT_RESULTS_FILE_PATH, index=False)

### NOTE: If you want to re-run the sentiment analysis, delete or archive the csv to create a blank one

# Sentiment Analysis
NOTE: Google gemini **currently** has a daily query limit of 1,500 requests per day.  As we have over 10,000 documents, the code will be designed to run over multiple days and pick up where we last left off.  After the code is run, the user will still need to manually download the csv and upload to github.

**Gemini Free Rate Limits**
*   15 RPM (requests per minute)
*   1 million TPM (tokens per minute)
*   1,500 RPD (requests per day)

In [7]:
# Set up Gemini. (API Key needs to be in your secrets)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

In [8]:
# Function to find the first empty row of the csv
def find_first_unique_id_with_empty_values(file_path, categories):
    """
    Finds the first unique ID where any of the specified columns have empty values in a CSV file.

    Args:
        file_path (str): The path to the CSV file.
        categories (list of str): List of column names to check for empty values.

    Returns:
        str: The first Unique_ID where any of the specified columns have empty values.
        None: If no such row is found.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Iterate over each row to find the first unique ID with any empty values in the specified columns
    for index, row in df.iterrows():
        if row[categories].isnull().any() or (row[categories] == '').any():
            return row['Unique_ID']

    return None  # Return None if no such row is found

# Test function
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
print(unique_id)

IR-62


In [9]:
# Helper function to get Gemini query function inputs
def get_gemini_inputs(text_df, unique_id):
    """
    Retrieves information from the DataFrame based on the unique ID and outputs company, source, headline, and text.

    Args:
        text_df (pd.DataFrame): The DataFrame containing the text data.
        unique_id (str): The unique ID to search for.

    Returns:
        tuple: A tuple containing company, source, headline, and text.
    """
    # Find the row with the specified unique ID
    row = text_df[text_df['Unique_ID'] == unique_id]

    # Extract the required information
    company = row['Ticker'].values[0]
    source = row['Source'].values[0]
    headline = row['Article Headline'].values[0]
    text = row['Article Text'].values[0]

    return company, source, headline, text

# Test function
company, source, headline, text = get_gemini_inputs(text_df, unique_id)
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: DVN

Source: Investment Research

Headline: Devon Energy Corporation

Text:
Stock Report Front| March 09, 2024 | NYSESymbol: DVN | DVN is in the S&P 500
Devon Energy Corporation
Recommendation Price 12-Mo. Target Price Report Currency Investment Style
BUY « « « « «
USD 46.16 (as of market close Mar 08, 2024) USD 51.00 USD Large-Cap Value
Equity Analyst Stewart Glickman, CFA
GICS Sector Energy Summary Devon Energy is one of the largest independent oil & gas exploration and production
Sub-Industry Oil and Gas Exploration and Production companies in the U.S.
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 56.19 - 40.47 Oper.EPS2024E USD 5.33 Market Capitalization[B] USD 29.29 Beta 2.20
Trailing 12-Month EPS USD 5.7 Oper.EPS2025E USD 5.80 Yield [%] 1.91 3-yr Proj. EPS CAGR[%] -7
Trailing 12-Month P/E 8.1 P/E on Oper.EPS2024E 8.66 Dividend Rate/Share USD 0.88 SPGMI's Quality Ranking B-
USD 10K Invested 5 Yrs Ago 21,575.0 

In [10]:
# Function to query Gemini
def query_gemini(company, source, headline, text, model):
    """
    Query Gemini to perform sentiment analysis on text from various sources about a company.

    Parameters:
    company (str): The name of the company the text is about.
    source (str): The source of the text. Valid values are "Investment Research", "ProQuest", "SEC Filings", "Earnings Call Presentations", "Earnings Call Q&A".
    headline (str): The headline or title of the text.
    text (str): The body of the text to be analyzed.
    model: The model object used to generate content and analyze the text.

    Returns:
    str: The sentiment analysis results for predefined categories in the specified format.

    The function constructs a prompt based on the source of the text and performs sentiment analysis on the given text using the provided model.
    It analyzes the content across multiple predefined categories, determining the sentiment (Positive, Neutral, Negative) for each category.
    If a category is not mentioned or relevant based on the text content, it is marked as 'Neutral'.
    The final output is summarized in a specified format.
    """

    # Source Variables
    if source == "Investment Research":
        text_source = "an analyst report"
        text_source2 = "the analyst report"
    elif source == "ProQuest":
        text_source = "a news article"
        text_source2 = "the news article"
    elif source == "SEC Filings":
        text_source = "an SEC filing"
        text_source2 = "the SEC filing"
    elif source == "Earnings Call Presentations":
        text_source = "an earnings call presentation"
        text_source2 = "the earnings call presentation"
    elif source == "Earnings Call Q&A":
        text_source = "an earnings call Q&A session"
        text_source2 = "the earnings call Q&A session"

    # Prompt
    prompt = f"""
Given the text from {text_source} about {company}, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

Before giving your answer, explain your reasoning and reference the article.
After going through all the categories, provide a summary in the following format:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Make sure to use plain text, do not bold or bullet the output summary.

The text from {text_source2} is below:
{headline}
{text}

Remember to summarize your final answers in the following format exactly:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Make sure to use plain text.
DO NOT bold or bullet the output summary.
    """

    # print(prompt)
    response = model.generate_content(prompt)
    return response.text

# Test function
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of DVN Analyst Report:

Here's a breakdown of the sentiment across the predefined categories, with reasoning and references to the text:

**Finance:**

- **Positive:** The report highlights DVN's strong financial performance, including a reduction in long-term debt, improved EBITDA, and a solid ratio of debt to EBITDA. The report also mentions DVN's ability to maintain a solid ratio in 2024 and their lack of major debt payments until late 2025.  
- **Reasoning:**  The report states "DVN’s long-term debt at year-end 2022 stood at $6.2 billion, down from $6.5 billion at year-end 2021. Relative to projected 2023 EBITDA, we calculate this is a ratio of 0.7x, an improvement from 1.95x at mid-2020."  It also notes "Barring a major step back in crude oil prices, we think DVN can maintain a solid ratio in 2024. DVN has no major milestone debt payments due until late 2025."
- **Reference:** Paragraph starting with "FINANCIAL TRENDS."

**Production:**

- **Neutral:** The re

In [11]:
# Function to parse text
def parse_sentiment(text, categories):
    """
    Parses a given text for specified categories and their sentiments.

    Args:
        text (str): The input text containing categories and their sentiments.
        categories (list of str): List of category names to look for in the text.

    Returns:
        dict or str: A dictionary with categories as keys and their corresponding sentiments as values,
                     or "Did not find all categories" if any sentiment is not Positive, Neutral, or Negative.
    """
    results = {}
    valid_sentiments = {"Positive", "Neutral", "Negative"}

    for category in categories:
        # Create a regex pattern to match the format "- Category - Sentiment"
        # The category name is escaped to handle any special characters
        pattern = rf"- {re.escape(category)} - (\w+)"

        # Search for the pattern in the text
        match = re.search(pattern, text)

        # If a match is found, extract the sentiment and add it to the results dictionary
        if match:
            sentiment = match.group(1)
            if sentiment not in valid_sentiments:
                return "Did not find all categories"
            results[category] = sentiment
        else:
            return "Did not find all categories"

    return results

# Test function
fail_text = """
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Bad
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral
"""

sentiment_dict = parse_sentiment(response, CATEGORIES)
fail_sentiment = parse_sentiment("fail_text", CATEGORIES)
print(sentiment_dict)
print(fail_sentiment)

{'Finance': 'Positive', 'Production': 'Neutral', 'Reserves / Exploration / Acquisitions / Mergers / Divestments': 'Negative', 'Environment / Regulatory / Geopolitics': 'Neutral', 'Alternative Energy / Lower Carbon': 'Positive', 'Oil Price / Natural Gas Price / Gasoline Price': 'Neutral'}
Did not find all categories


In [12]:
# Function to update the csv
def update_csv(file_path, unique_id, sentiment_dict):
    """
    Updates the columns of a CSV file based on the unique ID and sentiment dictionary.

    Args:
        file_path (str): The path to the CSV file.
        unique_id (str): The unique ID of the row to be updated.
        sentiment_dict (dict): A dictionary with categories as keys and their corresponding sentiments as values.

    Returns:
        None
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Find the index of the row with the specified unique ID
    row_index = df[df['Unique_ID'] == unique_id].index

    # Update the columns based on the sentiment dictionary
    for category, sentiment in sentiment_dict.items():
        df.loc[row_index, category] = sentiment

    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
    print(f"Row with Unique_ID '{unique_id}' has been updated.")

# Test function
update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

Row with Unique_ID 'IR-62' has been updated.


In [13]:
# Find the first empty row
start_time = time.time()
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
count = 0

# Iterate through the CSV using the functions
while unique_id:
    # Get gemini inputs
    company, source, headline, text = get_gemini_inputs(text_df, unique_id)

    # Query Gemini
    response = query_gemini(company, source, headline, text, model)

    # Parse text
    sentiment_dict = parse_sentiment(response, CATEGORIES)

    # Update the csv
    update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

    # Get an update every 10 rows
    count += 1
    if count % 10 == 0:
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        print(f"Iteration: {count}, Elapsed Time: {int(minutes)} minutes and {seconds:.2f} seconds")

    # Find the next empty row
    unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)

    # Find the next empty row
    unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)

    # Test print statements
    # print(unique_id)
    # print(f"Company: {company}\n")
    # print(f"Source: {source}\n")
    # print(f"Headline: {headline}\n")
    # print(f"Text:\n{text}")
    # print(response)
    # print(sentiment_dict)

Row with Unique_ID 'IR-63' has been updated.
Row with Unique_ID 'IR-64' has been updated.
Row with Unique_ID 'IR-65' has been updated.
Row with Unique_ID 'IR-66' has been updated.
Row with Unique_ID 'IR-67' has been updated.
Row with Unique_ID 'IR-68' has been updated.
Row with Unique_ID 'IR-69' has been updated.
Row with Unique_ID 'IR-70' has been updated.
Row with Unique_ID 'IR-71' has been updated.
Row with Unique_ID 'IR-72' has been updated.
Iteration: 10, Elapsed Time: 1 minutes and 50.73 seconds
Row with Unique_ID 'IR-73' has been updated.
Row with Unique_ID 'IR-74' has been updated.
Row with Unique_ID 'IR-75' has been updated.
Row with Unique_ID 'IR-76' has been updated.
Row with Unique_ID 'IR-77' has been updated.
Row with Unique_ID 'IR-78' has been updated.
Row with Unique_ID 'IR-79' has been updated.
Row with Unique_ID 'IR-80' has been updated.
Row with Unique_ID 'IR-81' has been updated.
Row with Unique_ID 'IR-82' has been updated.
Iteration: 20, Elapsed Time: 3 minutes and 

KeyboardInterrupt: 